In [1]:
import zipfile
import pandas as pd
from google.cloud import storage

In [2]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/cris/.gc/keys.json'

In [3]:
google_application_credentials = os.environ.get('GOOGLE_APPLICATION_CREDENTIALS')
print(google_application_credentials)

/home/cris/.gc/keys.json


In [33]:
# Initialize GCS client
client = storage.Client()

# Define GCS bucket and zip file path
bucket_name = 'cris-voter-data'
zip_file_path = 'voter_activity_20220828.zip'
destination_path = 'temp.zip'

try:
    # Get the bucket
    bucket = client.get_bucket(bucket_name)
    print("Bucket:", bucket)

    # Get the blob (file) from the bucket
    blob = bucket.blob(zip_file_path)
    print("Blob:", blob)

    # Define the destination path for the downloaded file
    destination_path = 'temp.zip'

    # Download the blob's content and write it to a file
    with open(destination_path, 'wb') as file:
        blob.download_to_file(file)
    
    print("Downloaded zip file to:", destination_path)

    # Check if download was successful
    if os.path.exists(destination_path):
        # Extract the zip file
        with zipfile.ZipFile(destination_path, 'r') as zip_ref:
            zip_ref.extractall('extracted_files')
        print("Successfully extracted zip file contents.")
    else:
        print("Failed to download zip file.")

except Exception as e:
    print("Error:", e)


Bucket: <Bucket: cris-voter-data>
Blob: <Blob: cris-voter-data, voter_activity_20220828.zip, None>
Downloaded zip file to: temp.zip
Successfully extracted zip file contents.


In [6]:
column_names = ['election_date', 'election_type', 'voter_id', \
                'tmp1', 'tmp2', 'tmp3', 'tmp4', 'tmp5', 'tmp6', 'tmp7', 'tmp8', \
                'zip', 'city', 'party_affiliation', \
                'tmp9', 'tmp10', \
                'ward', 'precinct', 'voter_status', \
                'tmp11', 'tmp12', 'tmp13', 'tmp14', 'tmp15', 'tmp16']

schema = {
  'election_date': 'object',
  'election_type': 'object',
  'voter_id': 'object',
  'tmp1': 'object',
  'tmp2': 'object',
  'tmp3': 'object',
  'tmp4': 'object',
  'tmp5': 'object',
  'tmp6': 'object',
  'tmp7': 'object',
  'tmp8': 'object',
  'zip': 'object',
  'city': 'object',
  'party_affiliation': 'object',
  'tmp9': 'object',
  'tmp10 ': 'object',
  'ward': 'object',
  'precinct': 'object',
  'voter_status': 'object',
  'tmp11': 'object',
  'tmp12': 'object',
  'tmp13': 'object',
  'tmp14': 'object',
  'tmp15': 'object',
  'tmp16': 'object'
}

In [7]:
# Read the text file into a DataFrame
df = pd.read_csv('extracted_files/001_voter_act.txt', delimiter='|', names=column_names, dtype=schema)
df['election_date'] = pd.to_datetime(df['election_date'])

In [8]:
df.dtypes

election_date        datetime64[ns]
election_type                object
voter_id                     object
tmp1                         object
tmp2                         object
tmp3                         object
tmp4                         object
tmp5                         object
tmp6                         object
tmp7                         object
tmp8                         object
zip                          object
city                         object
party_affiliation            object
tmp9                         object
tmp10                         int64
ward                         object
precinct                     object
voter_status                 object
tmp11                        object
tmp12                        object
tmp13                        object
tmp14                        object
tmp15                        object
tmp16                        object
dtype: object

In [9]:
print('n_rows = ', len(df))

n_rows =  237190


In [ ]:
# Read each text file into a DataFrame and write to Parquet format
for file_name in zip_ref.namelist():
    if file_name.endswith('.txt'):
        df = pd.read_csv('extracted_files/' + file_name, delimiter='|')
        # Process the data if needed
        # Write DataFrame to Parquet format
        df.to_parquet(file_name[:-4] + '.parquet', engine='pyarrow')

# Upload Parquet files to GCS
for file_name in zip_ref.namelist():
    if file_name.endswith('.parquet'):
        blob = bucket.blob('output/' + file_name[:-4] + '.parquet')
        blob.upload_from_filename(file_name[:-4] + '.parquet')

# Clean up temporary files
import os
os.remove(zip_file)
for file_name in zip_ref.namelist():
    if file_name.endswith('.parquet'):
        os.remove(file_name[:-4] + '.parquet')


AttributeError: 'NoneType' object has no attribute 'seek'